In [1]:
import json
import os
import pandas as pd
from src.mtbl_globals import DIR_EXTRACT, DIR_TRANSFORM

In [2]:
positions = {}
files = ['mtbl_c.json', 'mtbl_1b.json', 'mtbl_2b.json', 'mtbl_3b.json', 'mtbl_ss.json', 'mtbl_of.json', 'mtbl_dh.json', 'mtbl_rp.json', 'mtbl_sp.json']
for file in os.listdir(DIR_TRANSFORM):
    if file in files:
        file_path = os.path.join(DIR_TRANSFORM, file)
        with open(file_path, 'rb') as f:
            data = json.load(f)['data']
            
        pos = file.split('_')[1].split('.')[0]
            
        df = pd.DataFrame(data)
        positions[pos] = df

In [6]:
for pos, df in positions.items():
    print(pos)
    print(df.head())

3b
    ESPNID FANGRAPHSID   MLBID              name team     positions  proj_G  \
0    32801       13510  608070      Jose Ramirez  CLE      [3B, DH]     103   
1    42507       26289  683002  Gunnar Henderson  BAL      [3B, SS]     106   
2    33859       17350  646240     Rafael Devers  BOS          [3B]     102   
3    34982       18360  663586      Austin Riley  ATL          [3B]     104   
4  4722857       26323  668715     Spencer Steer  CIN  [1B, 3B, OF]      92   

   proj_PA  proj_AB  proj_H  ...  z_proj_SLG    z_total  pri_pos  R_shekels  \
0      443      393      56  ...    1.640995  10.506938       3B   3.824941   
1      463      407      55  ...    1.592364  10.144637       3B   4.182922   
2      442      390      56  ...    1.829778   8.985643       3B   3.760716   
3      451      402      59  ...    1.715630   8.412536       3B   3.887722   
4      395      347      47  ...    0.698752   6.611491       3B   2.818203   

   HR_shekels  RBI_shekels  SBN_shekels  OBP_sh

KeyboardInterrupt: 